In [1]:
import requests

response = requests.get('https://httpbin.org/ip')

print('Your IP is {0}'.format(response.json()['origin']))

Your IP is 40.89.248.181


In [2]:
import os
import azureml.core
from azureml.core.compute import ComputeTarget, DataFactoryCompute
from azureml.exceptions import ComputeTargetException
from azureml.core import Workspace, Experiment
from azureml.pipeline.core import Pipeline
from azureml.core.datastore import Datastore
from azureml.data.data_reference import DataReference
from azureml.pipeline.steps import DataTransferStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.83


In [3]:
import os, uuid
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

In [4]:
import os, uuid
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

try:
    print("Azure Blob storage v12 - Python quickstart sample")
    # Quick start code goes here
    AZURE_STORAGE_CONNECTION_STRING="DefaultEndpointsProtocol=https;AccountName=cscvteststorage;AccountKey=uef9P9ShgzQDAoIH058Gs/iKp/DRvVRmrBDR64OQJqZeV+n+fW0C9QhkCUNVZOYX73T9sFTgtKpoA1gpYSMGSg==;EndpointSuffix=core.windows.net"
    
    # Create the BlobServiceClient object which will be used to create a container client
    blob_service_client = BlobServiceClient.from_connection_string(AZURE_STORAGE_CONNECTION_STRING)
    
    # List containers
    print("\nListing containers...")
    containers_list = blob_service_client.list_containers()
    for c in containers_list:
        print("\t" + c.name)

except Exception as ex:
    print('Exception:')
    print(ex)

Azure Blob storage v12 - Python quickstart sample

Listing containers...
	test


In [5]:
container_client = blob_service_client.get_container_client("test")

In [6]:
blob_client = container_client.get_blob_client("annotations/43482-39665_U7wenFJaoMfpcncoohVmMA==..xml")

In [7]:
download_stream = blob_client.download_blob()

In [8]:
from pathlib import Path
import glob

# get resize factors
resizeDict = {}

newSize = 6000000 # bytes equal to 6MB
for filepath in glob.iglob('./images/*'):
    fileStem = Path(filepath).stem # removes file extension
    fileSize = os.stat(filepath).st_size
    resizeFactor = 1 - (fileSize - newSize)/fileSize
    resizeDict[fileStem] = resizeFactor

In [11]:
DEST_FILE = "test.xml"

In [12]:
import xml.etree.ElementTree as ET
from PIL import Image
from pathlib import Path

DEST_FILEIMG = "img1.jpg"
newSize = 6000000 # bytes equal to 6MB

async def read_content1(xml_file: str):

    tree = ET.parse(xml_file)
    root = tree.getroot()

    list_with_all_boxes = {}
    list_with_all_names = []
    i = 1

    for boxes in root.iter('object'):

        filename = root.find('filename').text
        
        with open(DEST_FILEIMG, "wb") as my_blob_img:
                blob_client_img = container_client.get_blob_client("images/" + filename)
                download_stream_img = blob_client_img.download_blob()
                my_blob_img.write(download_stream_img.readall())
                #imgpath = "images/" + filename
                #imgread = stream_test(imgpath)
                #my_blob_img.write(imgread)
        
        
        img = Image.open(DEST_FILEIMG)
        width, height = img.size
        
        #fileStem = Path(filename).stem # removes file extension
        #resizeFactor = resizeDict[fileStem]
        
        fileSize = os.stat(DEST_FILEIMG).st_size
        resizeFactor = 1 - (fileSize - newSize)/fileSize
        #resizeFactor = 1 - (fileSize - newSize)/fileSize
        if(fileSize > newSize):
          resizeFactor = 1 - (fileSize - newSize)/fileSize
          if(resizeFactor > 1): resizeFactor = 1
          newX = round(img.size[0] * resizeFactor)
          newY = round(img.size[1] * resizeFactor)

          img = img.resize((newX,newY), Image.ANTIALIAS)
          #image.save(dest + fileName, optimize=True, quality=
        
        
        
        objectname = boxes.find('name').text
        
        key = objectname

        ymin, xmin, ymax, xmax = None, None, None, None

        for box in boxes.findall("bndbox"):
            if(fileSize > newSize):
                xmin = int(box.find("xmin").text) * resizeFactor
                ymin = int(box.find("ymin").text) * resizeFactor
                xmax = int(box.find("xmax").text) * resizeFactor
                ymax = int(box.find("ymax").text) * resizeFactor
            else:
                xmin = int(box.find("xmin").text)
                ymin = int(box.find("ymin").text)
                xmax = int(box.find("xmax").text)
                ymax = int(box.find("ymax").text)
            
            # Normalize coordinates are given in the order: left, top, width, height.
            xNorm = xmin / width
            YNorm = ymin / height
            w = (xmax - xmin)/width
            h = (ymax - ymin)/height
            
        

        #list_with_single_boxes = [xmin, ymin, xmax, ymax, objectname]
        #list_with_single_boxes = [xmin, ymin, xmax, ymax, filename, objectname]
        list_with_single_boxes = [xNorm, YNorm, w, h, filename, objectname, img]
        #list_with_all_boxes.update(key = list_with_single_boxes)
        list_with_all_boxes[key + '_' + str(i)] = list_with_single_boxes
        list_with_all_names.append(objectname)
        
        i = i + 1

    return filename, list_with_all_boxes, list_with_all_names

#name, boxes = read_content("wasbs://test@cscvteststorage.blob.core.windows.net/annotations/43482-39665_U7wenFJaoMfpcncoohVmMA==..xml")
filename1, boxes1, names1 = await read_content1(DEST_FILE)

In [13]:
tagged_images_with_regions = []
#tagged_images_with_regions = {}

In [14]:
DEST_FILE1 = "test1.xml"

In [15]:
async def buildtags():
    blob_list = blob_service_client.get_container_client("test").list_blobs()
    for blob in blob_list:
        #print("\t" + blob.name)
        if("xml" in blob.name):
            print("\t" + blob.name)
            with open(DEST_FILE1, "wb") as my_blob:
                    blob_client = container_client.get_blob_client(blob.name)
                    download_stream = blob_client.download_blob()
                    my_blob.write(download_stream.readall())

            print("\t" + DEST_FILE1)


            filename, boxes, names = await read_content1(DEST_FILE1)
            strvalue = []
            #strvalue.append(filename)
            strvalue.append(boxes)
            tagged_images_with_regions.append(strvalue)
            import os
            if os.path.exists(DEST_FILE1):
                os.remove(DEST_FILE1)
            else:
                print("The file does not exist") 

In [16]:
await buildtags()

	annotations/43482-39665_U7wenFJaoMfpcncoohVmMA==..xml
	test1.xml
	annotations/43576-39678_188iAUvwvPXEmcigMeNosA==..xml
	test1.xml
	annotations/43711-39729_mZDyObN_8evlRxQl8CFVLw==..xml
	test1.xml
	annotations/43782-39754_6drOqUzJLvUKnIbchTOEuw==..xml
	test1.xml
	annotations/43943-39821_+I0ly82GF4wllMps+EeRQQ==..xml
	test1.xml
	annotations/44147-39851_djf2TFILSoiRAYtchKmyvw==..xml
	test1.xml
	annotations/45185-40263_qJ4tVUK7OfGfgzinhsWp8A==..xml
	test1.xml
	annotations/45596-40055_VDVrwCFj8T43JhcM9QYuGw==..xml
	test1.xml
	annotations/47955-45174_8_nYH9i7U9s6bUSEK1CJZw==..xml
	test1.xml
	annotations/47970-45040_fkTzx7SIsBGkohvhVs_PqA==..xml
	test1.xml
	annotations/47979-45244_qwLAiuNFWY5It7ACbj1z9Q==..xml
	test1.xml
	annotations/48014-45286_f9LS4AMHCkEAJhbFa5T+TA==..xml
	test1.xml
	annotations/48015-45013_lMVlMWnDrzFSftr5lCvt+A==..xml
	test1.xml
	annotations/48132-45358_VFix1fusTlfG5BKqw1zKkA==..xml
	test1.xml
	annotations/48187-44911_PYzS_ZKDjElG1tu4kB7fYw==..xml
	test1.xml
	annotatio

In [17]:
print(tagged_images_with_regions)

[[{'Insulator_1': [0.4378654970760234, 0.4838267543859649, 0.04952485380116959, 0.08854166666666667, '43482-39665_U7wenFJaoMfpcncoohVmMA==..jpg', 'Insulator', <PIL.MpoImagePlugin.MpoImageFile image mode=RGB size=5472x3648 at 0x7F02796A6630>], 'Insulator_2': [0.31048976608187134, 0.6008771929824561, 0.13998538011695907, 0.09210526315789473, '43482-39665_U7wenFJaoMfpcncoohVmMA==..jpg', 'Insulator', <PIL.MpoImagePlugin.MpoImageFile image mode=RGB size=5472x3648 at 0x7F02796A4DA0>], 'Insulator_3': [0.4928728070175439, 0.4492872807017544, 0.05299707602339181, 0.09375, '43482-39665_U7wenFJaoMfpcncoohVmMA==..jpg', 'Insulator', <PIL.MpoImagePlugin.MpoImageFile image mode=RGB size=5472x3648 at 0x7F02796A6438>], 'Insulator_4': [0.6482090643274854, 0.4342105263157895, 0.041301169590643276, 0.07510964912280702, '43482-39665_U7wenFJaoMfpcncoohVmMA==..jpg', 'Insulator', <PIL.MpoImagePlugin.MpoImageFile image mode=RGB size=5472x3648 at 0x7F0279711518>], 'Insulator_5': [0.5409356725146199, 0.462993421